In [13]:
import pandas as pd
import numpy as np

In [14]:
df2 = pd.read_csv('../scraped-data-2024/cluster2-html.csv')
df2 = pd.concat([df2, pd.read_csv('../scraped-data-2024/cluster2_5-pdf.csv')])

In [15]:
df2.replace('\xa0', np.nan, inplace=True)
df2.dropna(subset=["Pontuação Regata"], inplace=True)
df2['Punição'] = 0
df2['Descarte'] = 0

In [16]:
import re
import unidecode
import numpy as np

def process_string(s):
    # Remove espaços em branco extras
    s = str(s).strip()
    
    numbers = re.findall(r'\d+(?:\.\d+)?', s)
    if numbers == []:
        numbers.append(np.nan)
    
    has_parentheses = 1 if '(' in s or ')' in s else 0
    
    letters = ''.join(re.findall(r'[a-zA-Z]', s))
    if letters == '':
        letters = 0
    return numbers, has_parentheses, letters

In [17]:
df2['Punição'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[2])
df2['Descarte'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[1])
df2['Pontuação Regata'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[0][0])
df2["Nome Competidor"] = df2["Nome Competidor"] .apply(lambda x: unidecode.unidecode(x).upper())

In [18]:
df2.to_csv('../cleaned_data_2024/cluster_2.csv', index=False)

In [19]:
df3 = pd.read_csv('../scraped-data-2024/cluster3.csv')

In [20]:
df3["Nome Competidor"] = df3["Nome Competidor"].apply(lambda x: str(x).strip().replace('/', ', '))

In [21]:
# df3.to_csv('../cleaned_data_2024/cluster_3.csv', index=False)

In [22]:
df6 = pd.read_csv('../scraped-data-2024/cluster6-pdf.csv')

In [23]:
df6['Descarte'] = 0
df6.to_csv('../cleaned_data_2024/cluster6.csv')